In [5]:
# ----------
# Background
# 
# A robotics company named Trax has created a line of small self-driving robots 
# designed to autonomously traverse desert environments in search of undiscovered
# water deposits.
#
# A Traxbot looks like a small tank. Each one is about half a meter long and drives
# on two continuous metal tracks. In order to maneuver itself, a Traxbot can do one
# of two things: it can drive in a straight line or it can turn. So to make a 
# right turn, A Traxbot will drive forward, stop, turn 90 degrees, then continue
# driving straight.
#
# This series of questions involves the recovery of a rogue Traxbot. This bot has 
# gotten lost somewhere in the desert and is now stuck driving in an almost-circle: it has
# been repeatedly driving forward by some step size, stopping, turning a certain 
# amount, and repeating this process... Luckily, the Traxbot is still sending all
# of its sensor data back to headquarters.
#
# In this project, we will start with a simple version of this problem and 
# gradually add complexity. By the end, you will have a fully articulated
# plan for recovering the lost Traxbot.
# 
# ----------
# Part One
#
# Let's start by thinking about circular motion (well, really it's polygon motion
# that is close to circular motion). Assume that Traxbot lives on 
# an (x, y) coordinate plane and (for now) is sending you PERFECTLY ACCURATE sensor 
# measurements. 
#
# With a few measurements you should be able to figure out the step size and the 
# turning angle that Traxbot is moving with.
# With these two pieces of information, you should be able to 
# write a function that can predict Traxbot's next location.
#
# You can use the robot class that is already written to make your life easier. 
# You should re-familiarize yourself with this class, since some of the details
# have changed. 
#
# ----------
# YOUR JOB
#
# Complete the estimate_next_pos function. You will probably want to use
# the OTHER variable to keep track of information about the runaway robot.
#
# ----------
# GRADING
# 
# We will make repeated calls to your estimate_next_pos function. After
# each call, we will compare your estimated position to the robot's true
# position. As soon as you are within 0.01 stepsizes of the true position,
# you will be marked correct and we will tell you how many steps it took
# before your function successfully located the target bot.

# These import steps give you access to libraries which you may (or may
# not) want to use.
from robot import *
from math import *
from matrix import *
import random


# This is the function you have to write. The argument 'measurement' is a 
# single (x, y) point. This function will have to be called multiple
# times before you have enough information to accurately predict the
# next position. The OTHER variable that your function returns will be 
# passed back to your function the next time it is called. You can use
# this to keep track of important information over time.
def estimate_next_pos(measurement, OTHER = None):
    """Estimate the next (x, y) position of the wandering Traxbot
    based on noisy (x, y) measurements."""

    # AE: At first we just do a clumsy guess so that we have something for to return.
    xy_estimate = measurement
    
    # AE: If this is the first measurement, then just store it for the next
    # AE: call and return.
    # AE: 
    # AE: Otherwise start calculating
    if not OTHER:
        # AE: The way we'll be storing information is as follows:
        # AE: OTHER will be a list that contains the following lists: [prev_distance, prev_angle, prev_measurement]
        OTHER = []
        OTHER.append([0, 0, measurement])
    else:
        # AE: getting estimated distance and angle from previous run along with the previous measurement.
        prev_est_dst, prev_est_angle, prev_measurement = OTHER[0]
        cur_x, cur_y = measurement
        prev_x, prev_y = prev_measurement
        
        # AE: step size is easy- we already have a helper function for that.
        cur_est_distance = distance_between(measurement, prev_measurement)

        # AE: now based on right-angle triangle calculations we'll get the angle.
        # AE: It's worth noting that step_size is also the hypothenuse of a right triangle.
        #opposite = abs(cur_x - prev_x)
        opposite = cur_x - prev_x
        
        # AE: This is NOT the value by which it increased its angle last time.
        # AE: This is the total angle NOW. 
        # AE: We still want to know the angle increment.
        cur_est_angle = asin(opposite / cur_est_distance)
        
        # AE: We can only know the angle increment if we have accumulated 3 or more data points.
        if len(OTHER) > 2:
            angle_incr = cur_est_angle - prev_est_angle
            # AE: if we have a step size and the angle increment, then we can try to estimate
            new_est_angle = cur_est_angle + angle_incr
            new_est_angle = angle_trunc(new_est_angle)
            
            x_estimate = cur_x + cur_est_distance * cos(new_est_angle)
            y_estimate = cur_y + cur_est_distance * sin(new_est_angle)
            xy_estimate = (x_estimate, y_estimate)
            
            print("INFO: xy_estimate=", xy_estimate, " new_est_angle=", new_est_angle, " cur_est_distance=", cur_est_distance)
            print(" angle_incr=", angle_incr, " cur_est_angle=", cur_est_angle, " prev_est_angle=", prev_est_angle)
        
        # AE: ALways store previous measurement and current estimates.
        OTHER.insert(0, [cur_est_distance, cur_est_angle, measurement])
    
    # You must return xy_estimate (x, y), and OTHER (even if it is None) 
    # in this order for grading purposes.
    return xy_estimate, OTHER 

# A helper function you may find useful.
def distance_between(point1, point2):
    """Computes distance between point1 and point2. Points are (x, y) pairs."""
    x1, y1 = point1
    x2, y2 = point2
    return sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

# This is here to give you a sense for how we will be running and grading
# your code. Note that the OTHER variable allows you to store any 
# information that you want. 
def demo_grading(estimate_next_pos_fcn, target_bot, OTHER = None):
    localized = False
    distance_tolerance = 0.01 * target_bot.distance
    ctr = 0
    # if you haven't localized the target bot, make a guess about the next
    # position, then we move the bot and compare your guess to the true
    # next position. When you are close enough, we stop checking.
    while not localized and ctr <= 10: 
        ctr += 1
        measurement = target_bot.sense()
        position_guess, OTHER = estimate_next_pos_fcn(measurement, OTHER)
        target_bot.move_in_circle()
        true_position = (target_bot.x, target_bot.y)
        error = distance_between(position_guess, true_position)
        print("RES: g=", position_guess, " t=", true_position, " m=", measurement)
        if error <= distance_tolerance:
            print("You got it right! It took you ", ctr, " steps to localize.")
            localized = True
        if ctr == 10:
            print("Sorry, it took you too many steps to localize the target.")
    return localized

# This is a demo for what a strategy could look like. This one isn't very good.
def naive_next_pos(measurement, OTHER = None):
    """This strategy records the first reported position of the target and
    assumes that eventually the target bot will eventually return to that 
    position, so it always guesses that the first position will be the next."""
    if not OTHER: # this is the first measurement
        OTHER = measurement
    xy_estimate = OTHER 
    return xy_estimate, OTHER

# This is how we create a target bot. Check the robot.py file to understand
# How the robot class behaves.
test_target = robot(2.1, 4.3, 0.5, 2*pi / 34.0, 1.5)
test_target.set_noise(0.0, 0.0, 0.0)

#from result_visualization import *

#demo_grading(naive_next_pos, test_target)
demo_grading(estimate_next_pos, test_target)


RES: g= (2.1, 4.3)  t= (3.2618187593136714, 5.248776670511476)  m= (2.1, 4.3)
RES: g= (3.2618187593136714, 5.248776670511476)  t= (4.229518090705041, 6.394882252046046)  m= (3.2618187593136714, 5.248776670511476)
RES: g= (4.229518090705041, 6.394882252046046)  t= (4.970144154070997, 7.699287493626686)  m= (4.229518090705041, 6.394882252046046)
INFO: xy_estimate= (6.388429099657327, 8.187619156662356)  new_est_angle= 0.3315980553620026  cur_est_distance= 1.5000000000000002
 angle_incr= -0.18479956785822416  cur_est_angle= 0.5163976232202266  prev_est_angle= 0.7011971910784508
RES: g= (6.388429099657327, 8.187619156662356)  t= (5.4584758171066685, 9.117572439213015)  m= (4.970144154070997, 7.699287493626686)
INFO: xy_estimate= (6.942342473922045, 9.336980152822996)  new_est_angle= 0.14679848750378222  cur_est_distance= 1.5000000000000002
 angle_incr= -0.1847995678582222  cur_est_angle= 0.33159805536200443  prev_est_angle= 0.5163976232202266
RES: g= (6.942342473922045, 9.336980152822996) 

False